<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h1 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">M5.3 – معالجة البيانات سير عمل إعداد</h1><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">جزء من: <a href="https://github.com/OpenClimateScience/M5-Open-Science-for-Crops" dir="ltr">Open Climate Science for Crops &amp; Crop Conditions</a></p>
</div>

In [ ]:
import datetime
import glob
import os
import earthaccess
import numpy as np
import h5py
import xarray as xr
import rasterio
import rioxarray
import py4eos
import pyproj
from shapely.geometry import Polygon
from rasterio.warp import calculate_default_transform, reproject, Resampling
from matplotlib import pyplot
from tqdm import tqdm

auth = earthaccess.login()

LC_DIR = 'data/MCD12Q1'
VNP16_DIR = 'data/VNP16A2'
IMERG_DIR = 'data/IMERG'
OUTPUT_LC_FILENAME = 'data/processed/MODIS_MCD12Q1_Type5_cereal_croplands_h18v05_2023.tiff'
OUTPUT_VNP16_DIR = 'data/processed/VNP16_ET_and_PET'
OUTPUT_IMERG_DIR = 'data/processed'
TIME_PERIOD = ('2023-10-01', '2024-09-30')

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">نظرة عامة</h2><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">لحساب مؤشر رضا متطلبات المياه <span dir="ltr" style="unicode-bidi: isolate;">WRSI</span> سنحتاج إلى البيانات التالية:</p><ul>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">خريطة غطاء أرضي (لمساعدتنا على تحديد مواقع الأراضي الزراعية في منطقة الدراسة).</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><span dir="ltr" style="unicode-bidi: isolate;">PET</span> حساب النتح-التبخر الكامن.</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">الهطول.</li>
</ul><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">لكل عنصر مما سبق، لدى <span dir="ltr" style="unicode-bidi: isolate;">NASA</span> منتج يمكننا استخدامه:</p><ul>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><a href="https://doi.org/10.5067/MODIS/MCD12Q1.061" dir="ltr">مجموعة بيانات MODIS MCD12Q1</a> تصف صنف الغطاء الأرضي لكل بكسل <span dir="ltr" style="unicode-bidi: isolate;">500-m</span> على شبكة <span dir="ltr" style="unicode-bidi: isolate;">MODIS</span> الجيبية.</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">كما رأينا سابقًا، يوفر منتج <a href="https://doi.org/10.5067/MODIS/MOD16A2.006" dir="ltr">MODIS MOD16A2</a> تقديرات النتح-التبخر <span dir="ltr" style="unicode-bidi: isolate;">ET</span> و<span dir="ltr" style="unicode-bidi: isolate;">PET</span> كل <span dir="ltr" style="unicode-bidi: isolate;">8</span> أيام عالميًا.</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">واستخدمنا سابقًا أيضًا بيانات هطول <span dir="ltr" style="unicode-bidi: isolate;">NASA</span> من مهمة <span dir="ltr" style="unicode-bidi: isolate;">GPM</span> عبر مجموعة بيانات <a href="https://disc.gsfc.nasa.gov/datasets/GPM_3IMERGDF_06/summary" dir="ltr">IMERG-Final</a>.</li>
</ul><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>في هذا الدرس سنُجهّز مجموعات البيانات اللازمة لحساب</strong> <span dir="ltr" style="unicode-bidi: isolate;">WRSI</span>. والأهم أننا سنرى كيف نكتب قواعد <span dir="ltr" style="unicode-bidi: isolate;">Snakemake</span> التي تمكّن مستخدمين لاحقين من إعادة إنتاج خطوات معالجة البيانات بسرعة.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">على وجه التحديد، نريد أن نقوم بما يلي:</p><ol dir="rtl" style="direction: rtl; unicode-bidi: plaintext; text-align: right; padding-right: 1.2em;"><li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">إعادة أخذ العينات وإسقاط بيانات الغطاء الأرضي على شبكة <span dir="ltr" style="unicode-bidi: isolate;">EASE-Grid 2.0</span> بدقة <span dir="ltr" style="unicode-bidi: isolate;">1-km</span> لتسهيل مقارنتها مع مجموعات بيانات أخرى.</li></ol>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><hr></p><h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">اللازمة البيانات الحصول على</h2><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">سنستمر في استخدام مكتبة <span dir="ltr" style="unicode-bidi: isolate;">earthaccess</span> للوصول البرمجي إلى <span dir="ltr" style="unicode-bidi: isolate;">NASA Earthdata Search</span>. تذكّر أن الدالة <span dir="ltr" style="unicode-bidi: isolate;">earthaccess.search_data()</span> تحتوي على معامل <span dir="ltr" style="unicode-bidi: isolate;">bounding_box</span> للبحث عن البيانات ضمن إطار مستطيل محدد بإحداثيات خط العرض/خط الطول.</p><pre dir="ltr" style="unicode-bidi: isolate; text-align: left;"><code>help(earthaccess.search_data)</code></pre><pre dir="ltr" style="unicode-bidi: isolate; text-align: left;"><code>**bounding_box**: a tuple representing spatial bounds in the form
    `(lower_left_lon, lower_left_lat, upper_right_lon, upper_right_lat)`</code></pre><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">سنعرّف إطارنا <span dir="ltr" style="unicode-bidi: isolate;">bbox</span> لمنطقة الدراسة (جزء من شمال أفريقيا يتمركز حول الجزائر).</p>
</div>

In [ ]:
# The bounding box for our study area
bbox = (1.5, 34.0, 8.0, 37.0)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">منطقة دراستنا لِـ الغطاء الأرضي خريطة تنزيل</h3><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">عند تنزيل البيانات التي سنستخدمها، ينبغي أن نتذكر توثيق <strong>المُعرّف الرقمي</strong> <span dir="ltr" style="unicode-bidi: isolate;">DOI</span> لكل مجموعة بيانات. لا يساعد <span dir="ltr" style="unicode-bidi: isolate;">DOI</span> على الاستشهاد الصحيح وإعطاء الفضل للمؤلفين فحسب، بل يمكّن الآخرين أيضًا من تحديد مجموعة البيانات التي استخدمناها بدقة. بالنسبة لبيانات الغطاء الأرضي <span dir="ltr" style="unicode-bidi: isolate;">MODIS MCD12Q1</span> فإن <span dir="ltr" style="unicode-bidi: isolate;">DOI</span> هو:</p><ul>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">DOI: <span dir="ltr" style="unicode-bidi: isolate;">https://doi.org/10.5067/MODIS/MCD12Q1.061</span></li>
</ul>
</div>

In [ ]:
# Note that TIME_PERIOD refers to one of our global variables, defined at the top of the script
results = earthaccess.search_data(
    short_name = 'MCD12Q1',
    temporal = TIME_PERIOD,
    bounding_box = tuple(bbox))

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">هناك نتيجتان لأن الفترة الزمنية <span dir="ltr" style="unicode-bidi: isolate;">TIME_PERIOD</span> تمتد على سنتين.</p>
</div>

In [ ]:
len(results)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">عند التفكير في خط أنابيب معالجة بيانات قابل لإعادة الإنتاج، من المفيد تجنب إعادة تنزيل مجموعات بيانات كبيرة دون داعٍ. إذا كانت البيانات موجودة بالفعل، فلنقم بتفادي تنزيلها مرة أخرى.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">وبما أننا أنشأنا مجلدات منفصلة لكل مجموعة بيانات، يمكن لاختبار بسيط لوجود الملفات أن يحدد ما إذا كنا بحاجة إلى تنزيل البيانات. تذكّر أن الدالة <span dir="ltr" style="unicode-bidi: isolate;">glob.glob()</span> تسرد محتويات المجلد وفق نمط بحث، حيث تمثل النجمة <span dir="ltr" style="unicode-bidi: isolate;">*</span> محرفًا بديلًا يطابق أي سلسلة أحرف في اسم الملف.</p>
</div>

In [ ]:
# This is the directory we're searching
print(f'{LC_DIR}/*')

In [ ]:
# Only download the files once; i.e., if we haven't already downloaded any
if len(glob.glob(f'{LC_DIR}/*')) == 0:
    earthaccess.download(results, LC_DIR)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>سنستخدم مكتبة</strong> <span dir="ltr" style="unicode-bidi: isolate;">py4eos</span>، <a href="https://github.com/OpenClimateScience/M3-Open-Science-for-Water-Resources" dir="ltr">التي رأيناها سابقًا في الوحدة 3</a>، لفتح مجموعة بيانات الغطاء الأرضي <span dir="ltr" style="unicode-bidi: isolate;">MODIS MCD12Q1</span> لسنة <span dir="ltr" style="unicode-bidi: isolate;">2023</span>.</p>
</div>

In [ ]:
hdf = py4eos.read_file('data/MCD12Q1/MCD12Q1.A2023001.h18v05.061.2024252125305.hdf', platform = 'MODIS')
hdf

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">بعد ذلك نريد الحصول على بيانات الغطاء الأرضي ككائن <span dir="ltr" style="unicode-bidi: isolate;">rasterio</span>. نحن نستخدم الحقل <span dir="ltr" style="unicode-bidi: isolate;">&quot;LC_Type5&quot;</span> وهو واحد من عدة تصنيفات غطاء أرضي متاحة ضمن <span dir="ltr" style="unicode-bidi: isolate;">MODIS MCD12Q1</span>.</p>
</div>

In [ ]:
lc_raster = hdf.to_rasterio('LC_Type5', filename = '', driver = 'MEM')

In [ ]:
lc_map = lc_raster.read(1)

# Plot a preview
pyplot.imshow(lc_map, interpolation = 'nearest', vmax = 11)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">بيانات الغطاء الأرضي إسقاط</h3><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">نريد الآن إسقاط بيانات الغطاء الأرضي على شبكة <span dir="ltr" style="unicode-bidi: isolate;">EASE-Grid 2.0</span> بدقة <span dir="ltr" style="unicode-bidi: isolate;">1-km</span>. قد يكون من الصعب حساب العرض والارتفاع الجديدين لرستر بعد الإسقاط، لذلك نستخدم الدالة <a href="https://rasterio.readthedocs.io/en/latest/api/rasterio.warp.html#rasterio.warp.calculate_default_transform" dir="ltr">calculate_default_transform()</a> في الوحدة <span dir="ltr" style="unicode-bidi: isolate;">rasterio.warp</span> لحساب ذلك.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">في <span dir="ltr" style="unicode-bidi: isolate;">calculate_default_transform()</span> نحدد نظام الإحداثيات <span dir="ltr" style="unicode-bidi: isolate;">CRS</span> الحالي، و<span dir="ltr" style="unicode-bidi: isolate;">CRS</span> الجديد المطلوب، والعرض والارتفاع الحاليين، وحدود الرستر <span dir="ltr" style="unicode-bidi: isolate;">lc_raster.bounds</span>. مخرجات الدالة عبارة عن ثلاثية: نُسندها إلى ثلاثة متغيرات. آخر متغيرين هما <span dir="ltr" style="unicode-bidi: isolate;">width</span> و<span dir="ltr" style="unicode-bidi: isolate;">height</span> الجديدين، أما الأول <span dir="ltr" style="unicode-bidi: isolate;">new_transform</span> فيحمل معلومات مهمة عن الرستر الجديد.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">👈 ملاحظة: بلاطة <span dir="ltr" style="unicode-bidi: isolate;">MODIS</span> بدقة <span dir="ltr" style="unicode-bidi: isolate;">500-m</span> تكون عادة <span dir="ltr" style="unicode-bidi: isolate;">2400</span> صفًا × <span dir="ltr" style="unicode-bidi: isolate;">2400</span> عمودًا. إذا أعدنا أخذ العينات إلى <span dir="ltr" style="unicode-bidi: isolate;">1-km</span> (أي بكسلات أكبر بمرتين)، فسيكون عدد الصفوف والأعمدة نصف ذلك، أي الشكل <span dir="ltr" style="unicode-bidi: isolate;">(1200, 1200)</span>. أدناه نمرّر هذا الشكل إلى <span dir="ltr" style="unicode-bidi: isolate;">calculate_default_transform()</span>.</p>
</div>

In [ ]:
# NOTE: 6933 is the EPSG code for the global EASE-Grid 2.0 projection
new_transform, width, height = calculate_default_transform(
    lc_raster.crs, pyproj.CRS(6933), 1200, 1200, *lc_raster.bounds)

new_transform

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">الآن نحن جاهزون لإسقاط رستر الغطاء الأرضي باستخدام الدالة <span dir="ltr" style="unicode-bidi: isolate;">project()</span> من الوحدة <span dir="ltr" style="unicode-bidi: isolate;">rasterio.warp</span>. هناك نقاط مهمة في الشيفرة:</p><ul>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">نحتاج إلى فتح ملف إخراج للكتابة إليه؛ نستخدم <span dir="ltr" style="unicode-bidi: isolate;">rasterio.open()</span> ونمرّر نمط الملف <span dir="ltr" style="unicode-bidi: isolate;">&#x27;w+&#x27;</span> لنتمكن من الكتابة والقراءة.</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">نوع البيانات <span dir="ltr" style="unicode-bidi: isolate;">dtype</span> المستخدم هو <span dir="ltr" style="unicode-bidi: isolate;">np.uint8</span> (عدد صحيح غير موقّع <span dir="ltr" style="unicode-bidi: isolate;">8-bit</span>).</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">نستخدم <span dir="ltr" style="unicode-bidi: isolate;">Resampling.mode</span> (أخذ عينات بالأغلبية) لأن قيم الغطاء الأرضي تمثل فئات ولا ينبغي استيفاؤها كأعداد مستمرة.</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">ملف الإخراج <span dir="ltr" style="unicode-bidi: isolate;">OUTPUT_LC_FILENAME</span> سيكون <span dir="ltr" style="unicode-bidi: isolate;">GeoTIFF</span> لأننا استخدمنا الامتداد <span dir="ltr" style="unicode-bidi: isolate;">.tiff</span>.</li>
</ul>
</div>

In [ ]:
# See more at: https://rasterio.readthedocs.io/en/stable/topics/resampling.html

output_raster = rasterio.open(
    OUTPUT_LC_FILENAME, 'w+', count = 1, width = width, height = height,
    dtype = np.uint8, crs = pyproj.CRS(6933), transform = new_transform)

# Writing the new array to the file
reproject(
    source = rasterio.band(lc_raster, 1),
    destination = rasterio.band(output_raster, 1),
    resampling = Resampling.mode,
    src_nodata = 255,
    dst_nodata = 255)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">أخيرًا، نحوّل خريطة الغطاء الأرضي إلى خريطة ثنائية للأراضي الزراعية، حيث يشير <span dir="ltr" style="unicode-bidi: isolate;">1</span> إلى وجود أراضٍ زراعية و<span dir="ltr" style="unicode-bidi: isolate;">0</span> خلاف ذلك. وفقًا لـ <a href="https://lpdaac.usgs.gov/documents/1409/MCD12_User_Guide_V61.pdf" dir="ltr">دليل مستخدم MODIS MCD12Q1</a>، في تصنيف <span dir="ltr" style="unicode-bidi: isolate;">&quot;LC_Type5&quot;</span> يتم تمثيل الأراضي الزراعية أصلًا بالرقم <span dir="ltr" style="unicode-bidi: isolate;">7</span>.</p>
</div>

In [ ]:
lc_map_1km = output_raster.read(1)

# Create a binary array where Croplands==1, 0 for everything else
croplands = np.where(lc_map_1km == 7, 1, 0)
pyplot.imshow(croplands, interpolation = 'nearest')

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">سنقوم الآن بالكتابة فوق محتوى ملف <span dir="ltr" style="unicode-bidi: isolate;">GeoTIFF</span> الناتج الذي أنشأناه للتو، مستبدلين خريطة الغطاء الأرضي بخريطة الأراضي الزراعية الجديدة. يمكننا فعل ذلك لأن نوع البيانات والخصائص المكانية المطلوبة متطابقة.</p>
</div>

In [ ]:
output_raster.write(croplands, 1)
output_raster.close()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><hr></p><h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">أول <span dir="ltr" style="unicode-bidi: isolate;">Snakefile</span> لدينا</h2><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">بعد أن فهمنا كيفية معالجة بيانات الغطاء الأرضي، لنستخدم <span dir="ltr" style="unicode-bidi: isolate;">Snakemake</span> لكتابة قاعدة تجعل إعادة تنفيذ هذه الخطوة لاحقًا سهلة.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">لننشئ ملف <span dir="ltr" style="unicode-bidi: isolate;">Snakefile</span> يحتوي القاعدة التالية:</p><pre dir="ltr" style="unicode-bidi: isolate; text-align: left;"><code>rule download_data:
    params:
        dir_mcd12q1=&quot;data/MCD12Q1&quot;
    script:
        &quot;workflows/download_data.py&quot;</code></pre><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">هذا وصف عالي المستوى لأول مهمة لدينا: تنزيل بيانات الغطاء الأرضي. في السابق استخدمنا الكلمة المفتاحية <span dir="ltr" style="unicode-bidi: isolate;">shell</span> لتنفيذ أوامر سطر أوامر عامة. هنا بدلًا من <span dir="ltr" style="unicode-bidi: isolate;">shell</span> نستخدم <span dir="ltr" style="unicode-bidi: isolate;">script</span> للإشارة إلى أن <span dir="ltr" style="unicode-bidi: isolate;">Snakemake</span> ينبغي أن يشغّل سكربت <span dir="ltr" style="unicode-bidi: isolate;">Python</span> عند تنفيذ القاعدة. يعرف <span dir="ltr" style="unicode-bidi: isolate;">Snakemake</span> أن السكربت <span dir="ltr" style="unicode-bidi: isolate;">Python</span> بسبب امتداد الملف.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">👈 <strong>ملاحظة:</strong> مسارات الملفات هنا نسبية لمكان وجود ملف <span dir="ltr" style="unicode-bidi: isolate;">Snakefile</span>.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">👈 <strong>لاحظ الكلمة المفتاحية</strong> <span dir="ltr" style="unicode-bidi: isolate;">params</span>. هذه هي الطريقة التي نمرّر بها إعدادات (مثل مسارات مجلدات البيانات) إلى السكربت.</p>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h4 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">🏁 تحدٍّ: سكربت التنزيل كتابة</h4><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">تتطلب قاعدة <span dir="ltr" style="unicode-bidi: isolate;">Snakemake</span> أعلاه وجود سكربت <span dir="ltr" style="unicode-bidi: isolate;">Python</span> باسم <span dir="ltr" style="unicode-bidi: isolate;">download_data.py</span>. اكتب هذا السكربت اعتمادًا على الخطوات التي قمنا بها سابقًا لتنزيل بيانات الغطاء الأرضي.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>داخل سكربت</strong> <span dir="ltr" style="unicode-bidi: isolate;">Python</span> <strong>يمكنك الوصول إلى</strong> <span dir="ltr" style="unicode-bidi: isolate;">params</span>. كل زوج مفتاح/قيمة نكتبه داخل <span dir="ltr" style="unicode-bidi: isolate;">params</span> يصبح متاحًا داخل السكربت عبر <span dir="ltr" style="unicode-bidi: isolate;">snakemake.params</span>. على سبيل المثال عند تنزيل البيانات إلى مجلد محدد يمكننا كتابة:</p><pre dir="ltr" style="unicode-bidi: isolate; text-align: left;"><code>earthaccess.download(results, snakemake.params.dir_mcd12q1)</code></pre><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><a href="https://github.com/OpenClimateScience/M5-Open-Science-for-Crops/blob/main/notebooks/workflows/download_data_v1.py" dir="ltr">قارن نسخة download_data.py الخاصة بك مع سكربتنا</a></p><h4 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">🚩 <span style="color:red">انتبه</span></h4><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><span dir="ltr" style="unicode-bidi: isolate;">snakemake.params</span> يتم <strong>حقنه</strong> داخل الشيفرة عند تنفيذ السكربت بواسطة <span dir="ltr" style="unicode-bidi: isolate;">Snakemake</span>. لذلك <strong>لا</strong> تضف تعريفًا يدويًا لـ <span dir="ltr" style="unicode-bidi: isolate;">snakemake</span> داخل السكربت؛ سيتم توفيره تلقائيًا أثناء التشغيل.</p>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">أولى القاعدة تنفيذ</h3><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">لتشغيل قاعدة <span dir="ltr" style="unicode-bidi: isolate;">download_data</span> يجب أن نكون في المجلد نفسه الذي يوجد فيه ملف <span dir="ltr" style="unicode-bidi: isolate;">Snakefile</span>.</p><h4 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">🪟 🍎 🐧 جميع أنظمة التشغيل</h4><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">لتنفيذ القاعدة:</p><pre dir="ltr" style="unicode-bidi: isolate; text-align: left;"><code>snakemake --cores=1 download_data</code></pre>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><hr></p><h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">مهمة معالجة بيانات إضافة</h2><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">الخطوة التالية هي معالجة بيانات الغطاء الأرضي. لنكتب قاعدة جديدة في ملف <span dir="ltr" style="unicode-bidi: isolate;">Snakefile</span>:</p><pre dir="ltr" style="unicode-bidi: isolate; text-align: left;"><code>rule process_land_cover:
    input:
        &quot;data/MCD12Q1/MCD12Q1.A2023001.h18v05.061.2024252125305.hdf&quot;
    output:
        &quot;data/processed/MODIS_MCD12Q1_Type5_cereal_croplands_h18v05_2023.tiff&quot;
    script:
        &quot;workflows/process_land_cover.py&quot;</code></pre><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">معنى القاعدة واضح: ملف إدخال محدد يتحول إلى ملف إخراج محدد عبر سكربت <span dir="ltr" style="unicode-bidi: isolate;">Python</span> مُسمّى.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">👈 <a href="https://github.com/OpenClimateScience/M5-Open-Science-for-Crops/blob/main/notebooks/workflows/process_land_cover.py" dir="ltr">اطّلع على سكربت process_land_cover.py الذي كتبناه</a>. لقد نقلنا كل الشيفرة التي كتبناها أعلاه ووضعناها في سكربت واحد.</p><h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">الإخراج و الإدخال ملفات</h3><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">لاحظ أن القاعدة الجديدة تحدد ملفات الإدخال والإخراج صراحةً. هذا يساعد <span dir="ltr" style="unicode-bidi: isolate;">Snakemake</span> على تتبّع الاعتماديات وتشغيل ما يلزم فقط عند تغيّر البيانات أو الشيفرة.</p>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">بيانات <span dir="ltr" style="unicode-bidi: isolate;">ET</span> تنزيل</h2><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">سبق أن استخدمنا معادلة <span dir="ltr" style="unicode-bidi: isolate;">Hargreaves</span> (الوحدة 2) لحساب <span dir="ltr" style="unicode-bidi: isolate;">PET</span>، لكن يمكننا أيضًا الحصول على بيانات <span dir="ltr" style="unicode-bidi: isolate;">PET</span> من خوارزمية النتح-التبخر المعتمدة على الأقمار الصناعية لدى <span dir="ltr" style="unicode-bidi: isolate;">NASA</span> مثل <span dir="ltr" style="unicode-bidi: isolate;">MODIS MOD16</span> كما فعلنا في الوحدة 3.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">في هذا الدرس سنستخدم تقديرات <span dir="ltr" style="unicode-bidi: isolate;">PET</span> من منتج <a href="https://dx.doi.org/10.5067/VIIRS/VNP16A2GF.002" dir="ltr">VIIRS VNP16A2</a> الذي يستخدم خوارزمية <span dir="ltr" style="unicode-bidi: isolate;">MODIS MOD16</span> لكنه مبني على مستشعر <span dir="ltr" style="unicode-bidi: isolate;">VIIRS</span> الأحدث. تتوفر بيانات <span dir="ltr" style="unicode-bidi: isolate;">MODIS</span> و<span dir="ltr" style="unicode-bidi: isolate;">VIIRS</span> بنسخ «مملوءة الفجوات» <span dir="ltr" style="unicode-bidi: isolate;">GF</span>، لذلك الاسم القصير الذي سنستخدمه هنا هو <span dir="ltr" style="unicode-bidi: isolate;">&#x27;VNP16A2GF&#x27;</span>.</p>
</div>

In [ ]:
results = earthaccess.search_data(
    short_name = 'VNP16A2GF',
    temporal = TIME_PERIOD,
    bounding_box = tuple(bbox))

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">مثل كثير من منتجات <span dir="ltr" style="unicode-bidi: isolate;">MODIS</span> و<span dir="ltr" style="unicode-bidi: isolate;">VIIRS</span>، يأتي <span dir="ltr" style="unicode-bidi: isolate;">VNP16A2</span> على هيئة مركبات <span dir="ltr" style="unicode-bidi: isolate;">8</span>-أيام. يوجد <span dir="ltr" style="unicode-bidi: isolate;">46</span> فترة <span dir="ltr" style="unicode-bidi: isolate;">8</span>-أيام في سنة واحدة، وفترة الدراسة <span dir="ltr" style="unicode-bidi: isolate;">TIME_PERIOD</span> تمتد سنة واحدة. لكن عند فحص طول <span dir="ltr" style="unicode-bidi: isolate;">results</span> سنجد <span dir="ltr" style="unicode-bidi: isolate;">47</span> حبيبة <span dir="ltr" style="unicode-bidi: isolate;">granules</span>. ذلك لأن النتائج تتضمن فترة <span dir="ltr" style="unicode-bidi: isolate;">8</span>-أيام الأخيرة التي تبدأ في آخر يوم من سلسلتنا الزمنية. عمومًا من الجيد التحقق من طول نتائج <span dir="ltr" style="unicode-bidi: isolate;">earthaccess.search_data()</span>.</p>
</div>

In [ ]:
# Check that we're getting the expected number of granules
len(results)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">كما في السابق، سننزّل البيانات فقط إذا لم تكن قد نُزّلت من قبل.</p>
</div>

In [ ]:
# Only download the files once; i.e., if we haven't already downloaded any
if len(glob.glob(f'{VNP16_DIR}/*')) == len(results):
    earthaccess.download(results, VNP16_DIR)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">وسنستخدم <span dir="ltr" style="unicode-bidi: isolate;">py4eos</span> مرة أخرى لقراءة الملف—even وإن كان <span dir="ltr" style="unicode-bidi: isolate;">HDF5</span>—لأننا نحتاج طريقة سهلة لفتح البيانات كبيانات رستر يمكن التعامل معها.</p>
</div>

In [ ]:
hdf = py4eos.read_file('data/VNP16A2/VNP16A2GF.A2024001.h18v05.002.2025021191652.h5', platform = 'VIIRS')
hdf

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">يحتوي الملف على مجموعتي بيانات نهتم بهما:</p><ul>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><span dir="ltr" style="unicode-bidi: isolate;">PET_500m</span> يشير إلى بيانات <span dir="ltr" style="unicode-bidi: isolate;">PET</span>.</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><span dir="ltr" style="unicode-bidi: isolate;">ET_500m</span> يشير إلى بيانات <span dir="ltr" style="unicode-bidi: isolate;">ET</span> (وسنستخدمها لاحقًا).</li>
</ul><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">استنادًا إلى <a href="https://dx.doi.org/10.5067/VIIRS/VNP16A2GF.002" dir="ltr">دليل المستخدم</a> نعلم أن كلا المجموعتين لهما مجال صلاحية وأن القيم مُقاسة لتقليل حجم الملف. نستخدم <span dir="ltr" style="unicode-bidi: isolate;">numpy.where()</span> لإسناد <span dir="ltr" style="unicode-bidi: isolate;">np.nan</span> للقيم خارج مجال الصلاحية، ولإعادة تحجيم القيم الصحيحة بضربها في <span dir="ltr" style="unicode-bidi: isolate;">0.1</span>.</p>
</div>

In [ ]:
# Values greater than or equal to 32700 are NoData values
pet0 = hdf.get('PET_500m')
pet = np.where(np.abs(pet0) >= 32700, np.nan, pet0 * 0.1)

In [ ]:
pyplot.imshow(pet, interpolation = 'nearest')
pyplot.colorbar()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h4 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">🏁 تحدٍّ: سكربت التنزيل تحديث</h4><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">حدّث قاعدة <span dir="ltr" style="unicode-bidi: isolate;">download_data</span> وسكربت <span dir="ltr" style="unicode-bidi: isolate;">download_data.py</span> بحيث يقومان أيضًا بتنزيل بيانات <span dir="ltr" style="unicode-bidi: isolate;">VIIRS VNP16</span> الخاصة بـ <span dir="ltr" style="unicode-bidi: isolate;">ET</span> إلى المجلد الصحيح، اعتمادًا على المثال أعلاه.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><a href="https://github.com/OpenClimateScience/M5-Open-Science-for-Crops/blob/main/notebooks/workflows/download_data_v2.py" dir="ltr">قارن نسخة download_data.py الخاصة بك مع سكربتنا</a></p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">وبشكل خاص، لاحظ أننا أصبحنا نستخدم عدة قيم داخل <span dir="ltr" style="unicode-bidi: isolate;">params</span>:</p><pre dir="ltr" style="unicode-bidi: isolate; text-align: left;"><code>rule download_data:
    params:
        dir_mcd12q1=&quot;data/MCD12Q1&quot;,
        dir_vnp16a2=&quot;data/VNP16A2&quot;</code></pre><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">👈 عند وجود أكثر من عنصر داخل <span dir="ltr" style="unicode-bidi: isolate;">params</span> فإن الفاصلة ضرورية.</p>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><hr></p><h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><span dir="ltr" style="unicode-bidi: isolate;">ET</span> و<span dir="ltr" style="unicode-bidi: isolate;">PET</span> بيانات إسقاط وإعادة أخذ عينات</h2><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">كما في بيانات الغطاء الأرضي، نريد إسقاط بيانات <span dir="ltr" style="unicode-bidi: isolate;">ET</span> و<span dir="ltr" style="unicode-bidi: isolate;">PET</span> على شبكة <span dir="ltr" style="unicode-bidi: isolate;">EASE-Grid 2.0</span> بدقة <span dir="ltr" style="unicode-bidi: isolate;">1-km</span>. الأمر هنا أكثر تعقيدًا لأننا لا نملك قيمة <span dir="ltr" style="unicode-bidi: isolate;">NoData</span> واحدة؛ بل يجب اعتبار كل القيم الأكبر من <span dir="ltr" style="unicode-bidi: isolate;">32700</span> قيَمًا غير صالحة. لذلك سنستخدم <span dir="ltr" style="unicode-bidi: isolate;">read()</span> من كائن <span dir="ltr" style="unicode-bidi: isolate;">rasterio</span> للحصول أولًا على مصفوفة <span dir="ltr" style="unicode-bidi: isolate;">numpy</span> بعد إعادة أخذ العينات إلى <span dir="ltr" style="unicode-bidi: isolate;">1-km</span>، ثم نكتب هذه المصفوفة في كائن <span dir="ltr" style="unicode-bidi: isolate;">rasterio</span> جديد قبل الإسقاط.</p><h4 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">🏁 تحدٍّ</h4><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>اكتب دالة تقوم أولًا بإعادة أخذ العينات ثم بإسقاط بيانات</strong> <span dir="ltr" style="unicode-bidi: isolate;">VIIRS</span> الخاصة بـ <span dir="ltr" style="unicode-bidi: isolate;">ET</span> (أو <span dir="ltr" style="unicode-bidi: isolate;">PET</span>) على شبكة <span dir="ltr" style="unicode-bidi: isolate;">EASE-Grid 2.0</span> بدقة <span dir="ltr" style="unicode-bidi: isolate;">1-km</span>. استخدم ترويسة الدالة أدناه كنقطة بداية.</p><pre dir="ltr" style="unicode-bidi: isolate; text-align: left;"><code>def reproject_viirs(hdf, field, output_path = &#x27;&#x27;, driver = &#x27;MEM&#x27;):
    &#x27;&#x27;&#x27;
    Reprojects a VIIRS ET dataset to the global EASE-Grid 2.0.

    Parameters
    ----------
    hdf : py4eos.E</code></pre>
</div>

In [ ]:
def reproject_viirs(hdf, field, output_path = '', driver = 'MEM'):
    '''
    Reprojects a VIIRS ET dataset to the global EASE-Grid 2.0.

    Parameters
    ----------
    hdf : py4eos.EOSHDF4
        The EOSHDF4 instance connected to the VIIRS ET dataset
    field : str
        The name of the data variable, e.g., "ET_500m"
    output_path : str
        (Optional) The file path, if writing to a file on disk
    driver : str
        (Optional) The driver name, defaults to "MEM"

    Returns
    -------
    rasterio.io.DatasetWriter
    '''
    et_raster = hdf.to_rasterio(
        field, filename = '', driver = 'MEM', nodata = 32766., scale_and_offset = True)
    
    # First, resample the ET data to 1-km resolution
    arr = et_raster.read(out_shape = (1200, 1200), resampling = Resampling.average)
    arr = np.where(np.abs(arr) >= 32700, np.nan, arr)
    # We have to re-create the raster dataset, now at 1-km resolution
    et_raster_1km = rasterio.open(
        '', 'w+', driver = 'MEM', height = 1200, width = 1200,
        count = 1, dtype = np.float32, crs = et_raster.crs, 
        transform = et_raster.transform * et_raster.transform.scale(2)) # NOTE: Scaling to 1 km
    et_raster_1km.write(arr[0], 1)
    
    # Second, project the data onto a global EASE-Grid 2.0
    new_transform, width, height = calculate_default_transform(
        et_raster_1km.crs, pyproj.CRS(6933), 1200, 1200, *et_raster_1km.bounds)
    et_raster_ease2 = rasterio.open(
        output_path, 'w+', driver = driver, height = height, width = width,
        count = 1, dtype = np.float32, crs = pyproj.CRS(6933), transform = new_transform)
    reproject(
        source = rasterio.band(et_raster_1km, 1),
        destination = rasterio.band(et_raster_ease2, 1),
        resampling = Resampling.bilinear,
        src_nodata = np.nan, # Necessary so that missing data is interpolated
        dst_nodata = np.nan)
    return et_raster_ease2

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">لنعاين بيانات <span dir="ltr" style="unicode-bidi: isolate;">ET</span> من الملف الذي فتحناه سابقًا.</p>
</div>

In [ ]:
et_raster_ease2 = reproject_viirs(hdf, 'ET_500m')
img = et_raster_ease2.read(1)
pyplot.imshow(img, interpolation = 'nearest')
pyplot.colorbar()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">يبدو جيدًا. الآن نحتاج فقط إلى تطبيق هذه الدالة على كل الملفات التي نزّلناها. يمكننا فعل ذلك بحلقة <span dir="ltr" style="unicode-bidi: isolate;">for</span>. سنستخدم الدالة <span dir="ltr" style="unicode-bidi: isolate;">tqdm()</span> من حزمة <span dir="ltr" style="unicode-bidi: isolate;">tqdm</span> لإظهار شريط تقدم.</p>
</div>

In [ ]:
from tqdm import tqdm

# Get a list of all the files
file_list = glob.glob(f'{VNP16_DIR}/*')
file_list.sort()

et_data = []
pet_data = []
for filename in tqdm(file_list):
    # Read the date from the filename
    date = datetime.datetime.strptime(filename.split('/')[-1].split('.')[1], 'A%Y%j')
    # Convert the date to a YYYYMMDD string
    date_str = date.strftime('%Y%m%d')
    # Read in the input VNP16 granule
    hdf = py4eos.read_file(filename, platform = 'VIIRS')
    # Prepare the output filename, project the raster data
    et_filename = f'{OUTPUT_VNP16_DIR}/VNP16_ET_mm_8day-1_{date_str}.tiff'
    et0 = reproject_viirs(hdf, 'ET_500m', et_filename, driver = 'GTiff')
    pet_filename = f'{OUTPUT_VNP16_DIR}/VNP16_PET_mm_8day-1_{date_str}.tiff'
    pet0 = reproject_viirs(hdf, 'PET_500m', pet_filename, driver = 'GTiff')

    # Open the GeoTIFF files as xarray Datasets
    et = rioxarray.open_rasterio(et_filename)\
        .rename(band = 'time')\
        .assign_coords(time = [date])
    et_data.append(et)
    pet = rioxarray.open_rasterio(pet_filename)\
        .rename(band = 'time')\
        .assign_coords(time = [date])
    pet_data.append(pet)

# Convert from [mm 8day-1] to [mm day-1]
ds_et0 = xr.concat(et_data, dim = 'time').to_dataset(name = 'ET') / 8.0
ds_pet0 = xr.concat(pet_data, dim = 'time').to_dataset(name = 'PET') / 8.0
ds_et = xr.merge([ds_et0, ds_pet0])
ds_et.to_netcdf('data/processed/VNP16_ET_and_PET_mm_day-1.nc4')

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h4 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">🎯 أفضل الممارسات</h4><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">لاحظ أننا استدعينا <span dir="ltr" style="unicode-bidi: isolate;">file_list.sort()</span> أعلاه. لماذا؟</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">قد لا يكون ذلك مهمًا جدًا في هذا المثال، لكن عمومًا من المهم جدًا التأكد من أن قائمة الملفات مرتبة ترتيبًا مناسبًا قبل البدء بالمعالجة. إذا كانت الملفات تمثل سلسلة زمنية، فقد يكون من الضروري المعالجة وفق ترتيب زمني.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">إذا احتوى اسم الملف على تاريخ بصيغة سنة-شهر-يوم (مثل <span dir="ltr" style="unicode-bidi: isolate;">YYYYMMDD</span>) فإن استخدام <span dir="ltr" style="unicode-bidi: isolate;">sort()</span> سيضمن ترتيبًا زمنيًا لأن الفرز الأبجدي-الرقمي يطابق ترتيب التواريخ بهذه الصيغة.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>تأكيد:</strong> عندما تكون التواريخ في أسماء الملفات بصيغة <span dir="ltr" style="unicode-bidi: isolate;">YYYYMMDD</span> فإن الفرز الأبجدي-الرقمي باستخدام <span dir="ltr" style="unicode-bidi: isolate;">sort()</span> يعادل الفرز الزمني.</p>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><hr></p><h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><span dir="ltr" style="unicode-bidi: isolate;">Snakemake</span> إلى <span dir="ltr" style="unicode-bidi: isolate;">ET</span> معالجة إضافة</h2><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">نحتاج الآن إلى قاعدة جديدة في <span dir="ltr" style="unicode-bidi: isolate;">Snakemake</span> لمعالجة بيانات <span dir="ltr" style="unicode-bidi: isolate;">ET</span>. في السابق حددنا مجلد بيانات <span dir="ltr" style="unicode-bidi: isolate;">VIIRS VNP16</span> باستخدام <span dir="ltr" style="unicode-bidi: isolate;">params</span> داخل قاعدة <span dir="ltr" style="unicode-bidi: isolate;">download_data</span>. لكن عدة قواعد تحتاج معرفة مكان تخزين البيانات. هل يمكننا تجنب تكرار <span dir="ltr" style="unicode-bidi: isolate;">params</span> في كل قاعدة؟</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">نعم. هناك طريقتان على الأقل. أولًا، لأن <span dir="ltr" style="unicode-bidi: isolate;">Snakefile</span> هو في الأساس شيفرة <span dir="ltr" style="unicode-bidi: isolate;">Python</span>، يمكننا تعريف متغيرات عالمية ثم استخدامها داخل القواعد:</p><pre dir="ltr" style="unicode-bidi: isolate; text-align: left;"><code>DIR_MCD12Q1 = &quot;data/MCD12Q1&quot;
DIR_VNP16A2 = &quot;data/VNP16A2&quot;

rule download_data:
    params:
        dir_mcd12q1=DIR_MCD12Q1,
        dir_vnp16a2=DIR_VNP16A2
    script:
        &quot;workflows/download_data.py&quot;</code></pre><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>لكن سيكون أسهل عادةً إنشاء ملف إعدادات</strong> <span dir="ltr" style="unicode-bidi: isolate;">configuration</span> <strong>ليحمل المتغيرات العامة.</strong> ملف الإعدادات عادة يكون <span dir="ltr" style="unicode-bidi: isolate;">YAML</span> ويمكن تحميله داخل <span dir="ltr" style="unicode-bidi: isolate;">Snakemake</span> لاستخدامه عبر <span dir="ltr" style="unicode-bidi: isolate;">snakemake.config</span>.</p>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><hr></p><h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><span dir="ltr" style="unicode-bidi: isolate;">IMERG</span> من الهطول بيانات الحصول على</h2><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">بعد ذلك، لننزّل <a href="https://dx.doi.org/10.5067/GPM/IMERGDF/DAY/07" dir="ltr">بيانات الهطول من IMERG-Final</a>.</p>
</div>

In [ ]:
results = earthaccess.search_data(
    short_name = 'GPM_3IMERGDF',
    temporal = TIME_PERIOD)

In [ ]:
# Only download the files once; i.e., if we haven't already downloaded any
if len(glob.glob(f'{IMERG_DIR}/*')) == 0:
    earthaccess.download(results, IMERG_DIR)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">بيانات <span dir="ltr" style="unicode-bidi: isolate;">IMERG-Final</span> تأتي بصيغة <span dir="ltr" style="unicode-bidi: isolate;">netCDF4</span> والتي يمكن التعامل معها بسهولة أكبر في <span dir="ltr" style="unicode-bidi: isolate;">Python</span> باستخدام <span dir="ltr" style="unicode-bidi: isolate;">xarray</span>، كما ناقشنا في الوحدة 2.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">وتختلف بيانات <span dir="ltr" style="unicode-bidi: isolate;">IMERG-Final</span> عن بيانات الغطاء الأرضي <span dir="ltr" style="unicode-bidi: isolate;">MODIS</span> وبيانات <span dir="ltr" style="unicode-bidi: isolate;">VIIRS ET</span> في أن كل ملف يغطي الكرة الأرضية كاملة. لذلك نحتاج إلى قص كل ملف على منطقة دراستنا. طريقة سهلة هي استخدام الخاصية <span dir="ltr" style="unicode-bidi: isolate;">bounds</span> من رستر <span dir="ltr" style="unicode-bidi: isolate;">rasterio</span> الذي استخدمناه سابقًا. نستخدم <span dir="ltr" style="unicode-bidi: isolate;">shapely.geometry.Polygon</span> لتمثيل صندوق الحدود بشكل يفهمه <span dir="ltr" style="unicode-bidi: isolate;">rioxarray</span>. تذكّر أن <span dir="ltr" style="unicode-bidi: isolate;">rioxarray</span> مبني فوق <span dir="ltr" style="unicode-bidi: isolate;">xarray</span> ويُسهّل التعامل مع البيانات المكانية داخل <span dir="ltr" style="unicode-bidi: isolate;">xarray</span>.</p>
</div>

In [ ]:
# Getting the bounds of our VIIRS tile, for clipping other datasets
bb = et_raster_ease2.bounds
bounds = Polygon([
    (bb.left, bb.bottom), 
    (bb.left, bb.top),
    (bb.right, bb.top),
    (bb.right, bb.bottom)
])
bounds

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">لاحظ أن إحداثيات صندوق الحدود لدينا بوحدات أمتار شرقية/شمالية (انظر أدناه). ذلك لأن نظام الإحداثيات <span dir="ltr" style="unicode-bidi: isolate;">CRS</span> لبيانات <span dir="ltr" style="unicode-bidi: isolate;">ET</span> أصبح الآن شبكة <span dir="ltr" style="unicode-bidi: isolate;">EASE-Grid 2.0</span> العالمية. لذلك يجب التأكد من إسقاط بيانات الهطول إلى <span dir="ltr" style="unicode-bidi: isolate;">CRS</span> نفسه.</p>
</div>

In [ ]:
bb

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">يمكننا أيضًا حفظ هندسة صندوق الحدود إلى ملف لاستخدامها لاحقًا.</p>
</div>

In [ ]:
import shapely
import json

with open('data/processed/bounds.geojson', 'w') as file:
    json.dump(shapely.to_geojson(bounds), file)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">تذكّر أنه عندما تكون <span dir="ltr" style="unicode-bidi: isolate;">rioxarray</span> مثبتة، يمكننا الوصول إلى الخاصية <span dir="ltr" style="unicode-bidi: isolate;">.rio</span> لأي كائن <span dir="ltr" style="unicode-bidi: isolate;">xarray</span> لاستخدام دوال تعمل على البيانات المكانية. على سبيل المثال…</p>
</div>

In [ ]:
# Just an example
ds = xr.open_dataset('data/IMERG/3B-DAY.MS.MRG.3IMERG.20231001-S000000-E235959.V07B.nc4')
ds.rio

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">أصبحنا الآن جاهزين لمعالجة بيانات الهطول من <span dir="ltr" style="unicode-bidi: isolate;">IMERG-Final</span>. وبما أن كل ملف يمكن فتحه كـ <span dir="ltr" style="unicode-bidi: isolate;">xarray.Dataset</span>، فإن المعالجة ستكون سلسلة خطوات، كل خطوة تُنفّذ عبر استدعاء دالة (method) على الكائن.</p><ol dir="rtl" style="direction: rtl; unicode-bidi: plaintext; text-align: right; padding-right: 1.2em;"><li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">نرتّب أبعاد مصفوفة <span dir="ltr" style="unicode-bidi: isolate;">precipitation</span> لتكون بالترتيب: الزمن، خط العرض، خط الطول. يمكن ذلك باستدعاء الدالة <a href="https://docs.xarray.dev/en/stable/generated/xarray.Dataset.transpose.html" dir="ltr">transpose()</a>.</li><li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">نعيّن نظام الإحداثيات <span dir="ltr" style="unicode-bidi: isolate;">CRS</span> ونخبر <span dir="ltr" style="unicode-bidi: isolate;">xarray</span> ما هي الأبعاد التي تمثل محوري <span dir="ltr" style="unicode-bidi: isolate;">X</span> و<span dir="ltr" style="unicode-bidi: isolate;">Y</span>. نفعل ذلك عبر <span dir="ltr" style="unicode-bidi: isolate;">write_crs()</span> و<span dir="ltr" style="unicode-bidi: isolate;">set_spatial_dims()</span> (مع إضافة <span dir="ltr" style="unicode-bidi: isolate;">.rio</span> لأنهما من <span dir="ltr" style="unicode-bidi: isolate;">rioxarray</span>).</li><li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">بعد ذلك نقوم بإسقاط البيانات إلى شبكة عالمية <span dir="ltr" style="unicode-bidi: isolate;">EASE-Grid 2.0</span>، ثم نقصها على حدود منطقة الدراسة، ثم نُراكم الملفات في سجل زمني واحد.</li></ol>
</div>

In [ ]:
# NOTE: This may take up to 10 minutes to complete for 1 year's worth of data

file_list = glob.glob(f'{IMERG_DIR}/*.nc4')
file_list.sort()

stack = []
for filename in tqdm(file_list):
    ds = xr.open_dataset(filename)
    # Anytime we use the .rio attribute, we must have rioxarray installed
    ds_ease2 = ds[['precipitation']]\
        .transpose('time', 'lat', 'lon')\
        .rio.write_crs(4326)\
        .rio.set_spatial_dims('lon', 'lat')\
        .rio.reproject(pyproj.CRS(6933), resolution = 9000)\
        .rio.clip([bounds])
    stack.append(ds_ease2)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h4 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">🎯 أفضل الممارسات</h4><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">لاحظ أننا استدعينا مرة أخرى <span dir="ltr" style="unicode-bidi: isolate;">sort()</span> على قائمة الملفات <span dir="ltr" style="unicode-bidi: isolate;">file_list</span>. هنا هذا أمر أساسي لأننا نريد أن تكون الملفات التي نضعها في <span dir="ltr" style="unicode-bidi: isolate;">stack</span> مرتبة زمنياً.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>كما رأينا من قبل، يمكننا دمج مجموعات البيانات المنفصلة في كائن</strong> <span dir="ltr" style="unicode-bidi: isolate;">xarray.Dataset</span> <strong>واحد باستخدام الدالة</strong> <span dir="ltr" style="unicode-bidi: isolate;">concat()</span>.</p>
</div>

In [ ]:
ds_precip = xr.concat(stack, dim = 'time')

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><hr></p><h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">المشتقة البيانات منتجات حزم</h2><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">الكائن <span dir="ltr" style="unicode-bidi: isolate;">ds_precip</span> موجود فقط في ذاكرة الحاسوب. لا ننسى كتابته إلى القرص. هذا سهل في <span dir="ltr" style="unicode-bidi: isolate;">xarray</span>؛ وإذا كتبنا البيانات إلى ملف <span dir="ltr" style="unicode-bidi: isolate;">netCDF4</span> فسيتم حفظ السمات <span dir="ltr" style="unicode-bidi: isolate;">attributes</span> والبيانات الوصفية <span dir="ltr" style="unicode-bidi: isolate;">metadata</span> تلقائيًا.</p>
</div>

In [ ]:
ds_precip.to_netcdf(f'{OUTPUT_IMERG_DIR}/IMERG_precip_mm_day-1_for_study_area.nc4')

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">اخترت أن أضع وحدات الهطول (<span dir="ltr" style="unicode-bidi: isolate;">mm per hour</span>) في اسم الملف، لكن إن نسيت ذلك، فلاحظ أنها محفوظة أيضًا كصفة ضمن بيانات <span dir="ltr" style="unicode-bidi: isolate;">precipitation</span>.</p>
</div>

In [ ]:
ds_precip.precipitation.units

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><hr></p><h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><span dir="ltr" style="unicode-bidi: isolate;">Snakemake</span> إلى الهطول بيانات معالجة إضافة</h2><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">نحتاج إلى إضافة ما يلي إلى <span dir="ltr" style="unicode-bidi: isolate;">download_data.py</span>:</p><pre dir="ltr" style="unicode-bidi: isolate; text-align: left;"><code># download_data.py
results_imerg = earthaccess.search_data(
    short_name = &#x27;GPM_3IMERGDF&#x27;,
    temporal = TIME_PERIOD)
earthaccess.download(results_imerg, snakemake.config[&#x27;inputs&#x27;][&#x27;imerg&#x27;])</code></pre><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">كما نحتاج إلى إضافة المجلد الخاص ببيانات <span dir="ltr" style="unicode-bidi: isolate;">IMERG-Final</span> إلى ملف الإعدادات <span dir="ltr" style="unicode-bidi: isolate;">config.yaml</span>:</p><pre dir="ltr" style="unicode-bidi: isolate; text-align: left;"><code># config.yaml
inputs:
  mcd12q1: &quot;data/MCD12Q1&quot;
  vnp16a2: &quot;data/VNP16A2&quot;
  imerg: &quot;data/IMERG&quot;</code></pre><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">لمعالجة بيانات الهطول نحتاج إلى معرفة حدود بيانات <span dir="ltr" style="unicode-bidi: isolate;">ET</span> المُسقطة. أعلاه استخرجنا الحدود من ملف رستر <span dir="ltr" style="unicode-bidi: isolate;">ET</span> واحد من ناتج المعالجة؛ لذلك تتطلب معالجة الهطول أن تكون بيانات <span dir="ltr" style="unicode-bidi: isolate;">ET</span> و<span dir="ltr" style="unicode-bidi: isolate;">PET</span> قد عولجت أولًا. لربط القواعد بالترتيب في <span dir="ltr" style="unicode-bidi: isolate;">Snakemake</span> نجعل قاعدة الهطول تعتمد على ملف إدخال لا يتوفر إلا بعد اكتمال معالجة <span dir="ltr" style="unicode-bidi: isolate;">ET/PET</span>.</p>
</div>